In [4]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
from math import radians, cos, sin, asin, sqrt, isnan
import random
from ortools.sat.python import cp_model

In [5]:
#-----------------------------------EXTRACT THE DATA-----------------------------------

# users-melbcbd-generated.csv contains:
# •  Latitude-Longitude
# of the users in the Melbourne CBD area.
requests_path = '../eua-dataset/users/'
U = pd.read_csv(requests_path + 'users-test.csv') # test with 8 users
#R = pd.read_csv(requests_path + 'users-test-more-users.csv') # test with 14 users
#R = pd.read_csv(requests_path + 'users-melbcbd-generated.csv') # test with 816 users

# site-optus-melbCBD.csv contains:
# •  SiteID-Latitude-Longitude-Name-State-LicensingAreaID-PostCode-SitePrecision-Elevation-HCISL2
# of all Optus BS in Melbourne CBD area (edge-servers)
nodes_path = '../eua-dataset/edge-servers/'
#N = pd.read_csv(nodes_path + 'serverstest.csv') # test with 3 servers
N_src = pd.read_csv(nodes_path + 'serverstest_v2.csv') # test with 3 servers
N_dest = pd.read_csv(nodes_path + 'serverstest_v2.csv') # test with 3 servers
#N = pd.read_csv(nodes_path + 'serverstest-more-servers.csv') # test with 6 servers
#N = pd.read_csv(nodes_path + 'site-optus-melbCBD.csv') # test with 125 servers

In [6]:
#-----------------------------------INPUTS-----------------------------------


coordinates_R = pd.read_csv(requests_path + 'users-test.csv')


 




# Maximum allowed network delay for function f: phi_f
phi_f = [[0,0.06,0.06],[0.06,0,0.06],[0.06,0.06,0]]

In [7]:
#-----------------------------------INFRASTRUCTURED DATA-----------------------------------

# Cores on node j: U_j
U_j = [18,6,6]

In [8]:
#-----------------------------------MONITORED DATA-----------------------------------
# Network delay between nodes i and j
delta_ij = []

#Cores 
u_fj = [[2,2,2],[2,2,2],[2,2,2],[2,2,2]]

In [9]:
#-----------------------------------VARIABLES-----------------------------------


In [10]:
#-----------------------------------HAVERSINE-----------------------------------
def haversine(lon1, lat1, lon2, lat2):
  
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [31]:
#---------------------------------HEURISTIC MODEL---------------------------------
#----------------------------Input (STEP 1 - 4)-----------------------
# Incoming requests to node i
# Function(rows) x Node(column) [request/sec]
lambda_ri=[[1,0,0],[2,0,1],[1,1,1],[2,0,0]] 

# Amount of request received in time-slot
R=np.sum(lambda_ri)

# Identifies which user sent the request [U x R]
req_u =np.zeros([len(U),R])
req_u=[[1,0,0,0,0,0,0,0,0],
       [0,1,0,0,0,0,0,0,0],
       [0,0,1,0,0,0,0,0,0],
       [0,0,0,1,0,0,0,0,0],
       [0,0,0,0,1,0,0,0,0],
       [0,0,0,0,0,1,0,0,0],
       [0,0,0,0,0,0,1,0,0],
       [0,0,0,0,0,0,0,1,1]]

# Memory of function: m_f
m1 = 1
m2 = 2
m3 = 3
m4 = 4
m_f = [m1,m2,m3,m4]

# Set of requests conected to node n
S_uj = [] 
# Memory available in node j: M_j
M_j = [10,10,10]
#-------------------------Decision variables--------------
x_rj= np.zeros(shape=(len(N_dest),R))
c_fj=np.empty(shape=(len(m_f),len(N_dest)))
y_j=np.zeros(len(N_dest))
#-------------------------Sorting (Step 5)----------------
#Allocates at least one instance for each function (even with no incoming request)
if R == 0:
  for j in range(len(N_dest)):
    for f in range(len(m_f)):  
      if m_f[f] <M_j[j]:
        temp = c_fj[f]
        random_index = np.random.randint(0, len(temp))
        temp[random_index]=1
        c_fj[f]=temp
      else:
        c_fj[f][j]=0
  
  for f in range(len(m_f)):
    for j in range(len(N_dest)):
      if c_fj[f][j]==1:
        y_j[j]=1
#-------------------------Sorting (Step 6)----------------
# Show which requests are assigned to each function [F x R]
req_dist = np.zeros([len(m_f),R])

r = 0
while r<R:
    for i in range(len(N_src)):
        for f in range(len(m_f)):
            dif = lambda_ri[f][i] 
            while dif >0:
                req_dist[f][r]=1
                r=r+1
                dif = dif-1

# Matrix that assignes a function memory to each request [F x N]
m_request = np.empty((len(m_f),R))
for f in range(len(lambda_ri)):
  for r in range (R):
        m_request[f][r] = m_f[f]*req_dist[f][r]

# Sort the requests by their memory requirement --- returns position of the [] where request is found
m_index = []

for r in range (R):
  for f in range (len(m_f)):
    if m_request[f][r]!=0:
      m_index.append(m_request[f][r])
       
final_index=np.argsort(m_index)

#-------------------------Coverage servers-users (Step 7)----------------
#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(N_src), 0.03)

for i in range(len(N_src)):
  node_latitude = N_src.iloc[i]['LATITUDE']
  node_longitude = N_src.iloc[i]['LONGITUDE']
  temp = []
  for r in range(R):
    for u in range(len(U)):
      if req_u[u][r]==1:
        request_latitude = U.iloc[u]['Latitude']
        request_longitude = U.iloc[u]['Longitude']
        dist_geo = haversine(node_longitude, node_latitude, request_longitude, request_latitude)
        if dist_geo <= radius[i]:
          temp.append(1)
        
        else:
          temp.append(0)
       
  S_uj.append(temp)

